In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
import datetime
from tqdm.notebook import tqdm

## EXTRACT
Store datasets into DataFrames

In [2]:
csv_data = "../webscraping/miami_cl_webscrape.csv"
miami_cl_df = pd.read_csv(csv_data)

In [3]:
miami_cl_df

,Unnamed: 0,datetimes,hoods,prices,bedrooms,bathrooms,sqft,housing_type,laundry,parking,cats,dogs,furnished,titles,descriptions,links
0,0,2021-05-30 14:02,(Dadeland 2/2),"$1,900",2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 DADELAND: AMAZING UNIT Unique opportun...,\n\nQR Code Link to This Post\n\n\nPROP # 07K8...,https://miami.craigslist.org/mdc/apa/d/2-dadel...
1,1,2021-05-30 14:02,(Doral),"$2,557",2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,DEPÓSITO DE $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
2,2,2021-05-30 14:02,(Miami),"$2,241",2BR,2Ba,1106,apartment,w/d in unit,off-street parking,yes,yes,none,2 Bed 2 Bathroom Apartment,"\n\nQR Code Link to This Post\n\n\nOne, Two, a...",https://miami.craigslist.org/mdc/apa/d/miami-b...
3,3,2021-05-30 14:02,(Doral),"$3,399",3BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,"$500 deposit one,two,and three bedroom 24 hour...",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-5...
4,4,2021-05-30 14:01,(doral),"$1,978",1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,un mes GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2594,2594,2021-05-27 15:57,(5350 NW 84th Ave #413),"$7,500",3BR,3Ba,none,apartment,w/d in unit,detached garage,none,none,none,Beautiful turn-key furnished 3 bedroom unit wi...,\n\nQR Code Link to This Post\n\n\nBeautiful t...,https://miami.craigslist.org/mdc/apa/d/miami-b...
2595,2595,2021-05-27 15:54,(480 NE 31 St #4907),"$8,500",3BR,3Ba,none,house,w/d in unit,street parking,none,none,none,**Amazing Penthouse level 3 bedrooms and 3 bat...,\n\nQR Code Link to This Post\n\n\napartment f...,https://miami.craigslist.org/mdc/apa/d/miami-a...
2596,2596,2021-05-27 15:54,(Kendall 2/2),"$1,500",2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 2 Bedroom 2 Bathroom Unit Available KE...,\n\nQR Code Link to This Post\n\n\nPROP # 06K0...,https://miami.craigslist.org/mdc/apa/d/bedroom...
2597,2597,2021-05-27 15:54,(Fontainebleu 2/2.5),"$1,700",2BR,2.5Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2.5 FONTAINEBLEU: MAGNIFICENT UNIT Great...,\n\nQR Code Link to This Post\n\n\nPROP # 07K0...,https://miami.craigslist.org/mdc/apa/d/fontain...


## TRANSFORM
Clean DataFrame

In [4]:
miami_df = miami_cl_df.drop(columns=["Unnamed: 0"])
miami_df.head()

,datetimes,hoods,prices,bedrooms,bathrooms,sqft,housing_type,laundry,parking,cats,dogs,furnished,titles,descriptions,links
0,2021-05-30 14:02,(Dadeland 2/2),"$1,900",2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 DADELAND: AMAZING UNIT Unique opportun...,\n\nQR Code Link to This Post\n\n\nPROP # 07K8...,https://miami.craigslist.org/mdc/apa/d/2-dadel...
1,2021-05-30 14:02,(Doral),"$2,557",2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,DEPÓSITO DE $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
2,2021-05-30 14:02,(Miami),"$2,241",2BR,2Ba,1106,apartment,w/d in unit,off-street parking,yes,yes,none,2 Bed 2 Bathroom Apartment,"\n\nQR Code Link to This Post\n\n\nOne, Two, a...",https://miami.craigslist.org/mdc/apa/d/miami-b...
3,2021-05-30 14:02,(Doral),"$3,399",3BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,"$500 deposit one,two,and three bedroom 24 hour...",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-5...
4,2021-05-30 14:01,(doral),"$1,978",1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,un mes GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...


In [5]:
miami_df = miami_df.rename(columns = {"datetimes": "datetime", "hoods": "neighborhood", "prices": "rent",\
                                                "bedrooms": "br", "bathrooms": "ba", "housing_type": "housingType",\
                                                "titles": "title", "descriptions": "description", "links": "link"})
miami_df.head()

,datetime,neighborhood,rent,br,ba,sqft,housingType,laundry,parking,cats,dogs,furnished,title,description,link
0,2021-05-30 14:02,(Dadeland 2/2),"$1,900",2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 DADELAND: AMAZING UNIT Unique opportun...,\n\nQR Code Link to This Post\n\n\nPROP # 07K8...,https://miami.craigslist.org/mdc/apa/d/2-dadel...
1,2021-05-30 14:02,(Doral),"$2,557",2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,DEPÓSITO DE $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
2,2021-05-30 14:02,(Miami),"$2,241",2BR,2Ba,1106,apartment,w/d in unit,off-street parking,yes,yes,none,2 Bed 2 Bathroom Apartment,"\n\nQR Code Link to This Post\n\n\nOne, Two, a...",https://miami.craigslist.org/mdc/apa/d/miami-b...
3,2021-05-30 14:02,(Doral),"$3,399",3BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,"$500 deposit one,two,and three bedroom 24 hour...",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-5...
4,2021-05-30 14:01,(doral),"$1,978",1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,un mes GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...


In [6]:
miami_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2599 entries, 0 to 2598
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   datetime      2599 non-null   object
 1   neighborhood  2599 non-null   object
 2   rent          2599 non-null   object
 3   br            2599 non-null   object
 4   ba            2599 non-null   object
 5   sqft          2599 non-null   object
 6   housingType   2599 non-null   object
 7   laundry       2599 non-null   object
 8   parking       2599 non-null   object
 9   cats          2599 non-null   object
 10  dogs          2599 non-null   object
 11  furnished     2599 non-null   object
 12  title         2599 non-null   object
 13  description   2599 non-null   object
 14  link          2599 non-null   object
dtypes: object(15)
memory usage: 304.7+ KB


In [7]:
miami_df['datetime']

0       2021-05-30 14:02
1       2021-05-30 14:02
2       2021-05-30 14:02
3       2021-05-30 14:02
4       2021-05-30 14:01
              ...       
2594    2021-05-27 15:57
2595    2021-05-27 15:54
2596    2021-05-27 15:54
2597    2021-05-27 15:54
2598    2021-05-27 15:54
Name: datetime, Length: 2599, dtype: object

In [8]:
miami_df['datetime'] = pd.to_datetime(miami_df['datetime'])
miami_df['datetime']

0      2021-05-30 14:02:00
1      2021-05-30 14:02:00
2      2021-05-30 14:02:00
3      2021-05-30 14:02:00
4      2021-05-30 14:01:00
               ...        
2594   2021-05-27 15:57:00
2595   2021-05-27 15:54:00
2596   2021-05-27 15:54:00
2597   2021-05-27 15:54:00
2598   2021-05-27 15:54:00
Name: datetime, Length: 2599, dtype: datetime64[ns]

In [9]:
miami_df['neighborhood']

0                 (Dadeland 2/2)
1                        (Doral)
2                        (Miami)
3                        (Doral)
4                        (doral)
                  ...           
2594     (5350 NW 84th Ave #413)
2595        (480 NE 31 St #4907)
2596               (Kendall 2/2)
2597        (Fontainebleu 2/2.5)
2598               (Kendall 2/2)
Name: neighborhood, Length: 2599, dtype: object

In [10]:
miami_df['neighborhood'].value_counts()

 (Kendall 2/2)                          239
 (Miami Beach)                          188
 (Miami)                                154
 (SUNNY ISLES BEACH)                    148
 (Fontainebleu 2/2)                      98
                                       ... 
 (Sunny Isles Beach  Aventura Miami)      1
 (IVY BRICKELL)                           1
 (___ Aventura - Sunny Isles)             1
 (3271 SW 18th Ter Miami, FL)             1
 (BOCA RATON FL 33433)                    1
Name: neighborhood, Length: 380, dtype: int64

In [11]:
pd.set_option('display.max_rows', None)
miami_df['neighborhood'] = miami_df['neighborhood'].str.title()
miami_df['neighborhood'].value_counts()

 (Kendall 2/2)                                                     239
 (Miami)                                                           220
 (Miami Beach)                                                     199
 (Sunny Isles Beach)                                               158
 (Doral)                                                           128
 (Fontainebleu 2/2)                                                 98
 (Coral Gables)                                                     98
 (Brickell)                                                         78
 (Dadeland 2/2)                                                     75
 (Hammocks 2/2)                                                     69
 (North Miami)                                                      62
 (Kendall)                                                          46
 (Miami Beach , South Beach)                                        36
 (North Miami Beach)                                                35
 (Down

In [12]:
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('(', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(')', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(' 2/2.5', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(' 2/2', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(' 2/2', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Miami Beach , South Beach', 'South Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('South Beach Miami Beach', 'South Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('South Dadeland', 'Dadeland')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Edgewater Miami', 'Midtown / Edgewater')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(', FL', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(', Fl', '')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Brickell Miami', 'Brickell')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Miami South Beach', 'South Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Hammocks', 'Kendall')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('South Kendall', 'Kendall')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Downtown / Brickell', 'Brickell')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Kendall Lake', 'Kendall')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Deerwod', 'Deerwood')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Downtown Miami', 'Downtown')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Downtown / Miami River', 'Miami River')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Edgewater Miami', 'Edgewater')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Midtown / Wynwood', 'Wynwood')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('808 Brickell Key Dr #3105', 'Brickell')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Kkendall', 'Kendall')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Edgewater / Midtown', 'Midtown / Edgewater')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('West Hialeah', 'Hialeah')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Aventura - Sunny Isles Beach', 'Sunny Isles Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Little Havana / Dowtowwn', 'Little Havana')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Sunny Isles Beach - Aventura - Oceanfront Building', 'Sunny Isles Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('150 Sunny Isles Blvd #1-1102', 'Sunny Isles Beach')
miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Bricekll', 'Brickell')
miami_df['neighborhood'].value_counts()

 Kendall                                                         423
 Miami                                                           231
 Miami Beach                                                     204
 Sunny Isles Beach                                               167
 Dadeland                                                        130
 Doral                                                           128
 Coral Gables                                                    123
 Fontainebleu                                                    122
 Brickell                                                        101
 South Beach                                                      68
 North Miami                                                      62
 Downtown                                                         40
 North Miami Beach                                                35
 Hialeah                                                          28
 Aventura                         

In [13]:
broward_neighborhoods = ["Pembroke Pines", "Hollywood", "Fort Lauderdale", "Hallandale Beach"]

# rowstoDrop = [miami_df[~miami_df['neighborhood'].isin(broward_neighborhoods)] for a_df in broward_neighborhoods]
# rowstoDrop
# miami_df = miami_df.drop(rowstoDrop)
# miami_df['neighborhood'].value_counts()

indexes_to_drop = []

rows = miami_df.loc[miami_df['neighborhoods']]

miami_df = miami_df.drop([['neighborhoods'] for ['neighborhoods'] in miami_df if any((miami_df[['neighborhoods']] == i).all() for i in broward_neighborhoods)], 1)

# for i in tqdm(miami_df.index):
#     if [miami_df[(~miami_df['neighborhood'])].isin(broward_neighborhoods)]:
#         indexes_to_drop.append(i)
        
# miami_df.drop(indexes_to_drop).reset_index(level = 0, drop = True)

# miami_df.drop(miami_df.index[indexes_to_drop], inplace=True )

SyntaxError: can't assign to literal (<ipython-input-13-7b50e6264d19>, line 12)

In [ ]:
indexes_to_drop

In [ ]:
hood_col = miami_df['neighborhood'].value_counts().to_frame()
hood_col

In [ ]:
miami_df['rent']

In [ ]:
miami_df['rent'] = miami_df['rent'].str.replace(',', '')
miami_df['rent'] = miami_df['rent'].str.replace('$', '')
miami_df['rent']

In [ ]:
miami_df['rent'] = pd.to_numeric(miami_df['rent'])
miami_df['rent']

In [ ]:
miami_df['br']

In [ ]:
miami_df['br'] = miami_df['br'].str.replace('BR', '')
miami_df['br'] = pd.to_numeric(miami_df['br'])
miami_df['br']

In [ ]:
miami_df['ba']

In [ ]:
miami_df['ba'] = miami_df['ba'].str.replace('sharedBa', '')
miami_df['ba'] = miami_df['ba'].str.replace('Ba', '')
miami_df['ba'] = pd.to_numeric(miami_df['ba'])
miami_df['ba']

In [ ]:
miami_df['sqft']

In [ ]:
miami_df['sqft'] = miami_df['sqft'].str.replace('none', '')
miami_df['sqft'] = pd.to_numeric(miami_df['sqft'])
miami_df['sqft']

In [ ]:
miami_df['housingType'].value_counts()

In [ ]:
miami_df['laundry'].value_counts()

In [ ]:
miami_df['parking'].value_counts()

In [ ]:
miami_df['cats'].value_counts()

In [ ]:
miami_df['cats'] = miami_df['cats'].str.replace('none', 'no')
miami_df['cats']

In [ ]:
miami_df['dogs'].value_counts()

In [ ]:
miami_df['dogs'] = miami_df['dogs'].str.replace('none', 'no')
miami_df['dogs'].value_counts()

In [ ]:
miami_df['furnished'].value_counts()

In [ ]:
miami_df['furnished'] = miami_df['furnished'].str.replace('none', 'no')
miami_df['furnished'].value_counts()

In [ ]:
miami_df['title']

In [ ]:
miami_df['title'] = miami_df['title'].str.replace('●►  ', '')
miami_df['title'] = miami_df['title'].str.replace('*', '')
miami_df['title'] = miami_df['title'].str.replace('●', '')
miami_df['title']

In [14]:
miami_df['description']

0       \n\nQR Code Link to This Post\n\n\nPROP # 07K8...
1       \n\nQR Code Link to This Post\n\n\n*** UNFURNI...
2       \n\nQR Code Link to This Post\n\n\nOne, Two, a...
3       \n\nQR Code Link to This Post\n\n\n*** UNFURNI...
4       \n\nQR Code Link to This Post\n\n\n*** UNFURNI...
5       \n\nQR Code Link to This Post\n\n\n*** UNFURNI...
6       \n\nQR Code Link to This Post\n\n\n*** UNFURNI...
7       \n\nQR Code Link to This Post\n\n\n*** UNFURNI...
8       \n\nQR Code Link to This Post\n\n\n*** UNFURNI...
9       \n\nQR Code Link to This Post\n\n\nNewly renov...
10      \n\nQR Code Link to This Post\n\n\n*** UNFURNI...
11      \n\nQR Code Link to This Post\n\n\n\n*** UNFUR...
12      \n\nQR Code Link to This Post\n\n\nPROP # 07K9...
13      \n\nQR Code Link to This Post\n\n\nPROP # 06K2...
14      \n\nQR Code Link to This Post\n\n\nPROP # 07K3...
15      \n\nQR Code Link to This Post\n\n\n*** UNFURNI...
16      \n\nQR Code Link to This Post\n\n\nPROP # 07K0...
17      \n\nQR

In [15]:
miami_df['description'] = miami_df['description'].str.replace('\n\nQR Code Link to This Post\n\n\n', '')
miami_df['description'] = miami_df['description'].str.replace('\n', '')
miami_df['description'] = miami_df['description'].str.replace('PROP # 07K', '')
miami_df['description'] = miami_df['description'].str.replace('PROP # 06K', '')
miami_df['description']

0       80DADELAND: 2/2Unique opportunity to lease thi...
1       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
2       One, Two, and Three Bedroom Miami Apartments A...
3       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
4       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
5       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
6       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
7       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
8       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
9       Newly renovated large bedroom freshly painted ...
10      *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
11      *** UNFURNISHED THREE BEDROOM ****** MINIMUM O...
12      90DADELAND: 2/2Huge 2 bedroom with over sized ...
13      28HAMMOCKS: 2/2Rent this beautiful large 2/2 c...
14      38KENDALL: 2/2Spacious condo in the beautifull...
15      *** UNFURNISHED TWO BEDROOM ****** MINIMUM ONE...
16      08FONTAINEBLEU: 2/2RENT THIS LARGE 2/2 CONDO C...
17      IN-PER

In [18]:
miami_df['link'].value_counts()

https://miami.craigslist.org/mdc/apa/d/miami-beach-remodeled-and-fully/7328863548.html             2
https://miami.craigslist.org/mdc/apa/d/north-miami-beach-furnished-studio/7328504105.html          2
https://miami.craigslist.org/mdc/apa/d/miami-sls-brickell-pool-and-bayview/7316311783.html         1
https://miami.craigslist.org/mdc/apa/d/miami-beach-special-promotion-move-in/7328893835.html       1
https://miami.craigslist.org/mdc/apa/d/2-you-can-do-it-dadeland-huge-bedroom/7328745281.html       1
https://miami.craigslist.org/mdc/apa/d/miami-great-location-move-with-only-500/7319917332.html     1
https://miami.craigslist.org/mdc/apa/d/miami-fast-approvallow-depositpet/7320720158.html           1
https://miami.craigslist.org/mdc/apa/d/miami-spacious-bedroom-bath-with-many/7328488964.html       1
https://miami.craigslist.org/mdc/apa/d/miami-beach-great-studio-in-south-beach/7328912092.html     1
https://miami.craigslist.org/mdc/apa/d/north-miami-beach-fast-approval-only/7315335050.html